## Imports

In [1]:
# Enable automatic extension autoreloading
%load_ext autoreload
%autoreload 2

In [2]:
import torch

import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent.resolve()))
from model import get_model_with_unfrozen_layers

## Config

In [3]:
device = 4
device = torch.device(
    # f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"
    f"cuda:{device}" if torch.cuda.is_available() else "cpu"
)
device

device(type='cuda', index=4)

In [9]:
# Model
model_name = "lmsys/vicuna-7b-v1.3"

model = get_model_with_unfrozen_layers(
    model_name, device, num_layers_unfrozen=4,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
)
# model.eval()

memory_usage = model.pretrained_model.get_memory_footprint() / (1024 ** 3)
print(f"{memory_usage=:.2f} GB")

Loading policy model...

kwargs={'low_cpu_mem_usage': True, 'torch_dtype': torch.float16}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unfreezing bottom 4 layers...
Loaded subject model with 6,738,419,713 parameters.
Number of trainable params 940,613,633 parameters.
Model dtype: torch.float16

memory_usage=12.61 GB


In [10]:
model.training

True

In [11]:
# Num trainable parameters
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{num_trainable_params:,d}")

940,613,633


In [12]:
# model.pretrained_model

In [25]:
for name, p in model.pretrained_model.named_parameters():
    if p.requires_grad:
        print(f"{name}:      {p.numel():,d}")

model.layers.28.self_attn.q_proj.weight:      16,777,216
model.layers.28.self_attn.k_proj.weight:      16,777,216
model.layers.28.self_attn.v_proj.weight:      16,777,216
model.layers.28.self_attn.o_proj.weight:      16,777,216
model.layers.28.mlp.gate_proj.weight:      45,088,768
model.layers.28.mlp.down_proj.weight:      45,088,768
model.layers.28.mlp.up_proj.weight:      45,088,768
model.layers.28.input_layernorm.weight:      4,096
model.layers.28.post_attention_layernorm.weight:      4,096
model.layers.29.self_attn.q_proj.weight:      16,777,216
model.layers.29.self_attn.k_proj.weight:      16,777,216
model.layers.29.self_attn.v_proj.weight:      16,777,216
model.layers.29.self_attn.o_proj.weight:      16,777,216
model.layers.29.mlp.gate_proj.weight:      45,088,768
model.layers.29.mlp.down_proj.weight:      45,088,768
model.layers.29.mlp.up_proj.weight:      45,088,768
model.layers.29.input_layernorm.weight:      4,096
model.layers.29.post_attention_layernorm.weight:      4,096
mo

In [21]:
print(f"{sum(p.numel() for p in model.pretrained_model.lm_head.parameters() if p.requires_grad):,d}")

131,072,000


In [ ]:
print(f"{sum(p.numel() for p in model.pretrained_model.model.layers[30].parameters() if p.requires_grad):,d}")